# CATT diacritization error

## Download CATT

In [1]:
%cd catt

/home/auriza.akbar/nlp806-proj/catt


/home/auriza.akbar/.conda/envs/test/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


## Load datasets from HF

## Diacritize with CATT encoder-decoder

In [2]:
with open("../text_cla.txt") as f:
    text_cla = f.read().splitlines()
with open("../text_asc.txt") as f:
    text_asc = f.read().splitlines()
with open("../text_mdp.txt") as f:
    text_mdp = f.read().splitlines()

In [3]:
import torch
from ed_pl import TashkeelModel
from tashkeel_tokenizer import TashkeelTokenizer
from utils import remove_non_arabic

tokenizer = TashkeelTokenizer()
ckpt_path = 'models/best_ed_mlm_ns_epoch_178.pt'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('device:', device)

model = TashkeelModel(tokenizer, max_seq_len=1024, n_layers=3, learnable_pos_emb=False)
model.load_state_dict(torch.load(ckpt_path, map_location=device))
model.eval().to(device)

device: cuda


TashkeelModel(
  (transformer): Transformer(
    (encoder): Encoder(
      (emb): TransformerEmbedding(
        (tok_emb): TokenEmbedding(41, 512, padding_idx=0)
        (pos_emb): SinusoidalPositionalEncoding()
        (drop_out): Dropout(p=0.1, inplace=False)
      )
      (layers): ModuleList(
        (0-2): 3 x EncoderLayer(
          (attention): MultiHeadAttention(
            (attention): ScaleDotProductAttention(
              (softmax): Softmax(dim=-1)
            )
            (w_q): Linear(in_features=512, out_features=512, bias=False)
            (w_k): Linear(in_features=512, out_features=512, bias=False)
            (w_v): Linear(in_features=512, out_features=512, bias=False)
            (w_concat): Linear(in_features=512, out_features=512, bias=False)
          )
          (norm1): LayerNorm()
          (dropout1): Dropout(p=0.1, inplace=False)
          (ffn): PositionwiseFeedForward(
            (linear1): Linear(in_features=512, out_features=2048, bias=True)
         

In [4]:
from utils import strip_tashkeel

text_cla_ud = [strip_tashkeel(i) for i in text_cla]
text_cla_dx = model.do_tashkeel_batch(text_cla_ud)

text_asc_ud = [strip_tashkeel(i) for i in text_asc]
text_asc_dx = model.do_tashkeel_batch(text_asc_ud)

text_mdp_ud = [strip_tashkeel(i) for i in text_mdp]
text_mdp_dx = model.do_tashkeel_batch(text_mdp_ud)

100%|███████████████████████████████████████| 4116/4116 [05:03<00:00, 13.58it/s]


In [5]:
with open("../catt_cla.txt", "w") as f:
    for x in text_cla_dx: f.write(f"{x}\n")
with open("../catt_asc.txt", "w") as f:
    for x in text_asc_dx: f.write(f"{x}\n")
with open("../catt_mdp.txt", "w") as f:
    for x in text_mdp_dx: f.write(f"{x}\n")

## Compute DER

In [14]:
%%bash
echo ClArTTS
python compute_der.py ../text_cla.txt ../catt_cla.txt no
python compute_der.py ../text_cla.txt ../catt_cla.txt yes
echo ASC
python compute_der.py ../text_asc.txt ../catt_asc.txt no
python compute_der.py ../text_asc.txt ../catt_asc.txt yes
echo MDPC
python compute_der.py ../text_mdp.txt ../catt_mdp.txt no
python compute_der.py ../text_mdp.txt ../catt_mdp.txt yes

ClArTTS
Total DER: %2.053
Total WER: %7.926
Total mismatch samples: 0
-----------------------------------------------------------------------------------------
Total DER: %3.261
Total WER: %12.671
Total mismatch samples: 0
-----------------------------------------------------------------------------------------
ASC
Total DER: %3.442
Total WER: %14.843
Total mismatch samples: 1
-----------------------------------------------------------------------------------------
Total DER: %5.174
Total WER: %21.745
Total mismatch samples: 1
-----------------------------------------------------------------------------------------
MDPC
Total DER: %26.084
Total WER: %78.807
Total mismatch samples: 0
-----------------------------------------------------------------------------------------
Total DER: %31.294
Total WER: %87.761
Total mismatch samples: 0
-----------------------------------------------------------------------------------------


## Results

| Dataset | DER-NoCE | WER-NoCE | DER-CE | WER-CE |
|---------|----------|----------|--------|--------|
| ClArTTS | 2.05     | 7.93     | 3.26   | 12.67  |
| ASC     | 3.44     | 14.84    | 5.17   | 21.75  |
| MDPC    | 26.08    | 78.81    | 31.29  | 87.76  |